# Biologically Informed Climate Insights Through Animal-Borne Sensors


As climate change reshapes ecosystems, more precise and ecologically relevant measurements are needed. Traditional climate data are often limited by static, coarse, and sparse sampling, with indirect links to ecological impacts.
The MEOP consortium (MEOP stands for "Marine Mammals Exploring the Oceans Pole to Pole") brings together several national programmes to produce a comprehensive quality-controlled database of oceanographic data obtained in Polar Regions from instrumented marine mammals. Animal-borne sensors offer fine-scale, biologically tuned measurements of climatic conditions, enhancing ecological and climate forecasting. Millions of meteorological observations from over a thousand species have already been collected using these sensors.
This notebok explores how this growing dataset can bridge gaps in biodiversity and climate science, particularly in terrestrial environments, positioning tagged animals as key environmental sentinels and data providers for understanding changing ecosystems.

The tool uses the following product:

- OCEAN:ICE's ERDDAP's dataset (https://er1.s4oceanice.eu/erddap/tabledap/MEOP_Animal-borne_profiles.html)

The first code cell installs and imports into the notebook all the necessary libraries to run the code of the subsequent cells. The second one will generate two inputs to let the user decide the timerange to be considered when gathering the data from the ERDDAP MEOP dataset. The default values are 2020-01-01 as start date and 2020-02-05 as end date.  
Note: By increasing this timerange the time to gather the data and to elaborate the animations will also increase.

In [ ]:
# @title
!pip install cartopy

from ipywidgets import Dropdown, Text, Output, Layout, interactive
from IPython.display import HTML, clear_output
from matplotlib.animation import FuncAnimation
from IPython.display import Image
from matplotlib import colors as mcolors
from matplotlib.cm import ScalarMappable
from io import BytesIO
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib.collections as mcoll
import numpy as np
import pandas as pd
import requests
import datetime
import warnings
import time

warnings.filterwarnings('ignore')

In [ ]:
# @title
start_date = datetime.datetime.strptime('2020-01-01', "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")
end_date = datetime.datetime.strptime('2020-02-05', "%Y-%m-%d").strftime("%Y-%m-%dT00:00:00Z")

start_date_input = Text(
    value='2020-01-01',
    placeholder='2020-01-01',
    description='Start Date:',
    disabled=False
)

end_date_input = Text(
    value='2020-02-05',
    placeholder='2020-02-05',
    description='End Date:',
    disabled=False
)

def display_date_range(start, end):
    global start_date
    global end_date
    try:
        start_date = datetime.datetime.strptime(start, "%Y-%m-%d")
        end_date = datetime.datetime.strptime(end, "%Y-%m-%d")
        if start_date <= end_date:
            start_date = start_date.strftime("%Y-%m-%dT00:00:00Z")
            end_date = end_date.strftime("%Y-%m-%dT00:00:00Z")
    except ValueError:
        pass

interactive_widget = interactive(display_date_range, start=start_date_input, end=end_date_input)

display(interactive_widget)

interactive(children=(Text(value='2020-01-01', description='Start Date:', placeholder='2020-01-01'), Text(valu…

## Data gathering

The following code cell will download a list of platforms with data satisfying the time range specified above and with latitude less or equal than -55. This list of platforms is then displayed below.

In [ ]:
# @title
max_latitude = -55
all_platforms_south_url = f'https://er1.s4oceanice.eu/erddap/tabledap/MEOP_Animal-borne_profiles.csv?platform_code%2Ctime%2Clatitude&latitude%3C={max_latitude}&time%3E={start_date}&time%3C={end_date}&distinct()'
resp = requests.get(all_platforms_south_url)
df = pd.read_csv(BytesIO(resp.content), header=0, encoding='utf-8', dtype={'platform_code': str})
if 'time' in df.columns:
  df.drop(columns=['time'], inplace=True)
if 'latitude' in df.columns:
  df.drop(columns=['latitude'], inplace=True)
df = pd.DataFrame(df['platform_code'].unique(), columns=['platform_code'])
df = df[1:]
display(df)

,platform_code
1,85830
2,86562
3,89106
4,89172
5,89178
6,89202
7,89214
8,89286
9,89304
10,89328


In the next code cell the CTD data for the selected platforms will be gathered and displayed.

In [ ]:
# @title
all_data_traj = []
all_data_depth = []

for platform in df['platform_code']:
    platform_data_url = f'https://er1.s4oceanice.eu/erddap/tabledap/MEOP_Animal-borne_profiles.csv?platform_code%2Ctime%2CPRES%2CPSAL%2Clatitude%2Clongitude%2CTEMP&platform_code=%22{platform}%22&time%3E={start_date}&time%3C={end_date}'
    resp = requests.get(platform_data_url)

    if "Not Found: Your query produced no matching results. (nRows = 0)" in resp.text:
        continue

    df_platform = pd.read_csv(BytesIO(resp.content), encoding='utf-8', low_memory=False, dtype=str)
    df_platform = df_platform.iloc[1:]
    df_platform_filtered = df_platform.dropna(subset=['TEMP'])

    df_platform_temp = pd.DataFrame()

    df_platform_temp['PSAL'] = pd.to_numeric(df_platform_filtered['PSAL'], errors='coerce')
    df_platform_temp['PRES'] = pd.to_numeric(df_platform_filtered['PRES'], errors='coerce')
    df_platform_temp['TEMP'] = pd.to_numeric(df_platform_filtered['TEMP'], errors='coerce')
    df_platform_temp['latitude'] = pd.to_numeric(df_platform_filtered['latitude'], errors='coerce')
    df_platform_temp['longitude'] = pd.to_numeric(df_platform_filtered['longitude'], errors='coerce')

    df_platform_temp['time'] = pd.to_datetime(df_platform_filtered['time'], format='%Y-%m-%dT%H:%M:%SZ', errors='coerce')
    df_platform_temp['platform_code'] = df_platform_filtered['platform_code']

    df_platform_filtered = df_platform_temp

    df_daily_avg = df_platform_filtered.groupby([df_platform_filtered['platform_code'], df_platform_filtered['time'].dt.date]).agg({
        'TEMP': 'mean',
        'PSAL': 'mean',
        'latitude': 'mean',
        'longitude': 'mean'
    }).reset_index()

    all_data_traj.append(df_daily_avg)
    all_data_depth.append(df_platform_filtered)

if all_data_traj:
    data_df = pd.concat(all_data_traj, ignore_index=True)
    display(data_df)
else:
    print("No valid data found across platforms.")

,platform_code,time,TEMP,PSAL,latitude,longitude
0,85830,2020-01-01,-0.118417,32.814062,-61.468000,72.336567
1,85830,2020-01-02,-0.030854,32.793083,-60.742833,71.670400
2,85830,2020-01-03,-0.044109,32.766859,-59.918350,71.013575
3,85830,2020-01-04,0.346641,32.787813,-59.028125,70.506200
4,85830,2020-01-05,0.917047,32.822563,-58.185875,70.005800
...,...,...,...,...,...,...
584,wd13-420BAT-16,2020-01-03,-1.616500,34.545813,-66.555800,139.870200
585,wd13-420BAT-16,2020-01-04,-1.621188,34.547969,-66.646400,139.925350
586,wd13-420BAT-16,2020-01-05,-1.534156,34.552094,-66.639650,139.975900
587,wd13-420BAT-16,2020-01-06,-1.526188,34.545875,-66.660100,139.984400


## Trajectory plot

The code cell below will create and display a dropdown to let the user choose what parameter to process for the following plots.

In [ ]:
# @title
selected_param = 'Sea Water Temperature'

params_dropdown = Dropdown(
    options=['Sea Water Temperature', 'Practical Salinity'],
    value='Sea Water Temperature',
    description='Select the parameter:',
    layout=Layout(width='300px'),
    style={'description_width': 'initial'}
)

def on_change(change):
    global selected_param
    if params_dropdown.value:
        selected_param = params_dropdown.value

params_dropdown.observe(on_change, names='value')

display(params_dropdown)

Dropdown(description='Select the parameter:', layout=Layout(width='300px'), options=('Sea Water Temperature', …

The next code cell will visualize the trajectories of platforms over time, plotting the selected variable (either Sea Water Temperature or Practical Salinity) for each day. The map is centered on the Southern Hemisphere, and data points will be connected with lines to show the movement of each platform. The color of the lines represents the value for the selected variable, with a color bar provided for reference. The animation updates daily, highlighting how the platforms' trajectories evolve over time.

The data is first filtered to include only valid temperature readings, and then numeric fields such as temperature, salinity, pressure, latitude, and longitude are converted to appropriate types. For each platform and time, only the data from the shallowest depth is selected, ensuring that the temperature, salinity, latitude, and longitude values closest to the surface are used. This selection provides a meaningful surface-level representation. The grouped data is used to plot the daily average trajectory, improving efficiency while maintaining the accuracy of the visualized trends.

In [ ]:
# @title
%matplotlib agg

params_dict = {'Practical Salinity': 'PSAL', 'Sea Water Temperature': 'TEMP'}
param = params_dict[selected_param]

data_df['TEMP'] = pd.to_numeric(data_df['TEMP'], errors='coerce')
data_df['PSAL'] = pd.to_numeric(data_df['PSAL'], errors='coerce')
data_df['time'] = pd.to_datetime(data_df['time'], format='%Y-%m-%dT%H:%M:%SZ')
data_df['latitude'] = data_df['latitude'].astype(float)
data_df['longitude'] = data_df['longitude'].astype(float)

unique_days = sorted(data_df['time'].dt.date.unique())

norm = plt.Normalize(vmin=data_df[param].min(), vmax=data_df[param].max())
cmap = plt.get_cmap('coolwarm')
data_df['color'] = data_df[param].apply(lambda x: cmap(norm(x)))

fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.SouthPolarStereo()})


lines = []
status_text = ax.text(-170, -50, '', fontsize=12, color='black', transform=ccrs.PlateCarree())


def update_plot(day):
    ax.clear()
    ax.add_feature(cfeature.LAND, edgecolor='black')
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.set_extent([-180, 180, -90, -45], crs=ccrs.PlateCarree())
    
    
    current_data = data_df[data_df['time'].dt.date <= day]
    
    
    updated_artists = []
    
    
    for platform in current_data['platform_code'].unique():
        platform_data = current_data[current_data['platform_code'] == platform]
        for i in range(len(platform_data)-1):
            line, = ax.plot(
            [platform_data.iloc[i]['longitude'], platform_data.iloc[i+1]['longitude']],
            [platform_data.iloc[i]['latitude'], platform_data.iloc[i+1]['latitude']],
            color=platform_data.iloc[i]['color'], marker='.', linewidth=2.5,
            transform=ccrs.PlateCarree(), alpha=0.7
            )
            updated_artists.append(line)
            
    
    status_text = ax.text(-170, -50, f'Current Day: {day}, Data Points: {len(current_data)}',
    fontsize=12, color='black', transform=ccrs.PlateCarree())
    updated_artists.append(status_text)
    
    
    return updated_artists


ani = FuncAnimation(fig, update_plot, frames=unique_days, interval=250, repeat=True)
ani.save('animation.gif', writer='pillow', fps=4)
Image(filename='animation.gif')

## Pressure plot

The next code cell will display the DataFrame containing the data used for the final plot, which analyzes the selected parameter based on depth.

In [ ]:
# @title
if all_data_depth:
    data_df_pres_concat = pd.concat(all_data_depth, ignore_index=True)
    display(data_df_pres_concat)
else:
    print("No valid data found across platforms.")

,PSAL,PRES,TEMP,latitude,longitude,time,platform_code
0,32.453,4.0,0.333,-61.6209,72.5402,2020-01-01 04:20:00,85830
1,32.453,10.0,0.323,-61.6209,72.5402,2020-01-01 04:20:00,85830
2,32.433,20.0,-0.242,-61.6209,72.5402,2020-01-01 04:20:00,85830
3,32.411,44.0,-0.845,-61.6209,72.5402,2020-01-01 04:20:00,85830
4,32.447,50.0,-1.417,-61.6209,72.5402,2020-01-01 04:20:00,85830
...,...,...,...,...,...,...,...
184539,34.493,126.0,-1.781,-66.0269,142.2741,2020-01-01 22:10:00,wd13-911-18
184540,34.503,150.0,-1.731,-66.0269,142.2741,2020-01-01 22:10:00,wd13-911-18
184541,34.520,170.0,-1.840,-66.0269,142.2741,2020-01-01 22:10:00,wd13-911-18
184542,34.528,200.0,-1.829,-66.0269,142.2741,2020-01-01 22:10:00,wd13-911-18


The last code cell will visualize platform data at different pressure levels over time. The data points for each pressure level (binned in increments of 10 dbar) are plotted based on the selected parameter. The color of the data points represents the value of the selected parameter, with a color bar provided for reference.

This plot complements the previous trajectory plot, allowing users to visualize another fundamental component of Conductivity-Temperature-Depth (CTD) profiles.  Averaging is applied to the selected parameter. A second depth scale on the left visualizes the pressure levels, and the animation updates the map by showing data points for different pressure bins, enabling users to observe how conditions change with depth.

In [ ]:
# @title
pres_bin_size = 10


# Prepare data
data_df_pres = data_df_pres_concat.copy()
param = params_dict[selected_param]


norm = plt.Normalize(vmin=data_df[param].min(), vmax=data_df[param].max())
data_df_pres['PRES'] = data_df_pres['PRES'].astype(float)
data_df_pres['TEMP'] = data_df_pres['TEMP'].astype(float)
data_df_pres['PSAL'] = data_df_pres['PSAL'].astype(float)
data_df_pres['time'] = pd.to_datetime(data_df_pres['time'])


depth_bins = np.arange(0, data_df_pres['PRES'].max() + pres_bin_size, pres_bin_size)
data_df_pres['PRES'] = pd.cut(data_df_pres['PRES'], bins=depth_bins, labels=depth_bins[:-1], right=False)
data_df_pres['PRES'] = data_df_pres['PRES'].astype(float)


cmap = plt.get_cmap('coolwarm')
data_df_pres['color'] = data_df_pres[param].apply(lambda x: cmap(norm(x)))


# Create figure and axes
fig, ax = plt.subplots(figsize=(10, 8), subplot_kw={'projection': ccrs.SouthPolarStereo()})
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.set_extent([-180, 180, -90, -45], crs=ccrs.PlateCarree())


sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = fig.colorbar(sm, ax=ax, orientation='vertical', fraction=0.025, pad=0.1)
if param == 'TEMP':
    cbar.set_label('Sea Water Temperature (°C)', fontsize=12)
elif param == 'PSAL':
    cbar.set_label('Practical Salinity (psu)', fontsize=12)


unique_pres = sorted(data_df_pres['PRES'].unique())


# Depth axis for highlighting
ax_depth = fig.add_axes([0.1, 0.1, 0.03, 0.8])
ax_depth.set_ylim(0, data_df_pres['PRES'].max())
ax_depth.invert_yaxis()
ax_depth.set_xticks([])
ax_depth.set_yticks(np.arange(0, data_df_pres['PRES'].max() + 100, 100))
ax_depth.set_ylabel('Pressure (dbar)', fontsize=12)
ax_depth.plot([0.5, 0.5], [0, data_df_pres['PRES'].max()], color='black', lw=2)
highlight_bin = ax_depth.scatter([], [], color='black', s=100, zorder=5)


# Function to update frames
def update_plot(pres):
    if pres >= unique_pres[-1] or pres <= unique_pres[0]:
        print('\r' + ' ' * 50, end='', flush=True)
    else:
        print(f'\rUpdating for pressure: {int(pres)}/{int(unique_pres[-1])}', end='', flush=True)
    
    
    ax.clear()
    ax.add_feature(cfeature.LAND, edgecolor='black')
    ax.add_feature(cfeature.OCEAN)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=':')
    ax.set_extent([-180, 180, -90, -45], crs=ccrs.PlateCarree())
    
    
    current_data = data_df_pres[data_df_pres['PRES'] == pres]
    ax.scatter(current_data['longitude'], current_data['latitude'],
    c=current_data['color'], transform=ccrs.PlateCarree(), alpha=0.7, s=10)
    
    
    ax.set_title(f'Pressure Level: {int(pres)} dbar')
    highlight_bin.set_offsets([[0.5, pres]])


# Create animation
ani = FuncAnimation(fig, update_plot, frames=unique_pres, interval=500, repeat=True)


# Save as GIF
ani.save('pressure_animation.gif', writer='pillow', fps=4)


# Display in notebook
Image(filename='pressure_animation.gif')

### Additional resources

The Python libraries that have been used in this notebook are:
- [cartopy](https://scitools.org.uk/cartopy/docs/latest/)
- [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/)
- [requests](https://requests.readthedocs.io/en/latest/)
- [numpy](https://numpy.org/)
- [pandas](https://pandas.pydata.org/)
- [matplotlib](https://matplotlib.org/)